# Projet numérique : Câble sous-marin

*Auteurs : Valentin Fenoux et Mathis Boutrouelle*

## Questions théoriques

1.

2.

3.

4.

## Implémentation

In [1]:
# Chargement de dépendance

import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
# Discrétisation

A = 0
B = 500
N = 101
Delta = (B - A)/(N - 1)
discretization_indexes = np.arange(N)
discretization = discretization_indexes*Delta

In [3]:
# Paramètres du modèle

mu = -5
a = 50
sigma2 = 12

In [ ]:
# Données

observation_indexes = [0, 20, 40, 60, 80, 100]
depth = np.array([0, -4, -12.8, -1, -6.5, 0])